In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import mmap
import random
import pickle
from tqdm import tqdm

device = 'mps' if torch.backends.mps.is_available() else 'cpu'

batch_size = 32 # How many sets of blocks can be evaluated at the same time (the more, the more gpu intensive)
block_size = 128 # Size of the blocks used to predict the next token
max_iters = 50 # Amount of iterations in training loop
learning_rate = 3e-4 # Experiment with this and find the one that has the best performance + quality over time
# Some good values to test out: 3e-3, 3e-4, 1e-3, 1e-4
eval_iters = 100
n_embd = 384 # Length of vector for each item in the vocab (amnt of info to store per letter)
n_layer = 4 # Amount of blocks/layers to run sequentually
n_head = 4 # Amount of heads to run in parallel in MultiHeadAttention
dropout = 0.2 # Drop 20% of our neurons to prevent overfitting

In [2]:
chars = ""
with open("vocab.txt", "r", encoding="utf-8") as f:
    text = f.read()
    chars = sorted(list(set(text)))
    
vocab_size = len(chars)

In [3]:
string_to_int = { ch: i for i, ch in enumerate(chars) }
int_to_string = { i: ch for i, ch in enumerate(chars) }

def encode(s):
    return [string_to_int[c] for c in s]

def decode(l):
    return "".join([int_to_string[i] for i in l])

In [4]:
# Memory map for using small snippets of text from a single file of any size without having to open the whole file
def get_random_chunk(split):
    filename = "train_split.txt" if split == "train" else "val_split.txt"

    with open(filename, "rb") as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)

            # Decode the block to a string, ignoring any invalid byte sequence
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')

            # Train and test splits
            data = torch.tensor(encode(decoded_block), dtype=torch.long)

    return data

def get_batch(split):
    data = get_random_chunk(split)
    
    # Get 4 (batch_size) random integers which represent the start indices for our blocks
    # (ensure that we have enough characters in sequence hence len(data) - block_size)
    ix = torch.randint(len(data) - block_size, (batch_size,))

    # Get a tensor with 4 blocks
    x = torch.stack([data[i:i+block_size] for i in ix])
    # Same blocks but then offset by 1
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    # Send to GPU, meaning they will be processed in parallel
    x, y = x.to(device), y.to(device)
    
    return x, y

In [5]:
# PyTorch will not use gradients in this block (reduce computation costs)
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [10]:
class Head(nn.Module):
    """ one head of self-attention """
    
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        # Registering tril ahead of time to reduce training time + costs and increase performance (reduce overhead by initting beforehand)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step (block size?), channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x) # B,T,hs
        q = self.query(x) # B,T,hs
        # compute attention scores ("affinities")
        # Scaling by 1/sqrt(length of row in keys/queries matrix) to prevent exploding
        weights = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) DOT PRODUCT (B, hs, T) -> (B, T, T) -> Apply the scaling
        # Apply a masked_fill to prevent looking into the future
        weights = weights.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        # Apply softmax
        weights = F.softmax(weights, dim=-1) # B, T, T
        # Apply dropout
        weights = self.dropout(weights)
        # Perform the weighted aggregation of the values, matrix multiply with value for weights!
        value = self.value(x) # B, T, hs
        out = weights @ value # B, T, T @ B, T, hs -> B, T, hs
        return out
        

class MultiHeadAttention(nn.Module):
    """ Multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        # Heads that will be running in parallel in isolation
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        # Add more learnable params to help it learn more about the text
        self.projection = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Align heads by (B, T, C) C dimension (feature dimension)
        
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.projection(out))
        return out

class FeedForward(nn.Module):
    """ A simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

# Decoder block
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        # Number of features (n_embd) each head will capture in our multihead attention
        head_size = n_embd // n_head
        self.self_attention = MultiHeadAttention(n_head, head_size)
        self.feed_forward = FeedForward(n_embd)
        self.layer_norm1 = nn.LayerNorm(n_embd)
        self.layer_norm2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.self_attention(x)
        x = self.layer_norm1(x + y)
        y = self.feed_forward(x)
        x = self.layer_norm2(x + y)
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.positioning_embedding_table = nn.Embedding(block_size, n_embd)

        # Decoder blocks/layers
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])

        # Create a layernorm after processing of the decoder blocks
        self.layernorm_final = nn.LayerNorm(n_embd)
        self.languagemodelling_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    # Helps initialize our weights properly, no need to understand what is going on, just know it will make learning better
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02) # std = Standard deviation
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        
        
    def forward(self, index, targets=None):
        B, T = index.shape
        
        # index and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.positioning_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb
        
        # Feed the token embeddings + position encodings into our network!
        x = self.blocks(x) # (B,T,C)

        # Finalize + get the probabilities
        x = self.layernorm_final(x) # (B,T,C)
        logits = self.languagemodelling_head(x) # (B, T, vocab_size)
        
        if targets is None:
            loss = None
        else:
            # Batch (size), Time (sequence of integers we know), Channel (vocab size)
            B, T, C = logits.shape # Unpack dimensions
            # B & T not as important here, so we blend them together, as long as logits + targets have same batch
            # and time, we should be fine. We are more focused on vocab size (C) here
            # What we're doing here is change the shape of the tensor to the way cross_entropy expects, which is B, C, T rather than B, T, C
            logits = logits.view(B*T, C) # Reshape dimensions
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # Index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C) as we only care about the last token, hence -1 in the 2nd (T) dimension
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index
    
model = GPTLanguageModel(vocab_size)

print("loading model parameters...")
with open("model-01.pkl", "rb") as f:
    model = pickle.load(f)
m = model.to(device)
print("loaded successfully!")

loading model parameters...
loaded successfully!


In [15]:
# Create a PyTorch optimizer
# AdamW = optimizer with weight decay
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training loop
for iter in tqdm(range(max_iters)):
    # Current iteration nth of eval_iters
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, validation loss: {losses['val']:.3f}")
    
    # sample batch data
    xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    # Zero gradient.... not sure about this
    optimizer.zero_grad(set_to_none=True)
    
    # And don't fully understand the next 2 but apparently it's part of the learning cycle
    loss.backward()
    optimizer.step()
print(loss.item())

with open('model-01.pkl', 'wb') as f:
    pickle.dump(model, f)
print("model saved")

  0%|                              | 0/50 [00:00<?, ?it/s]

step: 0, train loss: 2.420, validation loss: 2.367


100%|█████████████████████| 50/50 [00:13<00:00,  3.69it/s]


2.6981072425842285
model saved


In [9]:
# Output of generated text with 1 newline character as the "context"
# Context is just a newline character index (torch.zeros(1,1))
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

RuntimeError: The size of tensor a (64) must match the size of tensor b (65) at non-singleton dimension 2